# Bank of America Stock Prediction with Prophet $BAC 💰

Install necessary packages/libraries: torch, transformers, pandas, numpy, yahoo finance, and any other required dependencies.

In [ ]:
pip install torch transformers pandas numpy

In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from prophet import Prophet

In [ ]:
# import historical data from yfinance stock ticker 'BAC'

bac = yf.download('BAC', start='2020-01-01', end='2024-07-29')

In [ ]:
#view columns
print(bac.columns)

In [ ]:
#view data types
bac.dtypes

In [ ]:
# This will move the index to a column since yfinance does not include a date column
bac.reset_index(inplace=True) 

In [ ]:
#quick view of data
print(bac.head())

# prepare our data by turning our date and adj close columns into ds and y

In [ ]:
bac['ds'] = pd.to_datetime(bac['Date'])
bac['y'] = bac['Adj Close']

In [ ]:
# Select and display only 'ds' and 'y' columns
bac2 = bac[['ds', 'y']]
print(bac2.head())

In [ ]:
#verify changes
bac2

In [ ]:
bac

# add stock indicators that would help with analysis like moving averages, trading volume, and technical indicators.

In [ ]:
bac['SMA_50'] = bac['Close'].rolling(window=50).mean()
bac['SMA_200'] = bac['Close'].rolling(window=200).mean()
bac['ma50'] = bac['Close'].rolling(window=50).mean()
bac['ma200'] = bac['Close'].rolling(window=200).mean()
bac['Volume'] = bac['Volume']
bacind = bac.dropna()
bacind

In [ ]:
# Calculate Relative Strength Index (RSI)
def RSI(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    RS = gain / loss
    return 100 - (100 / (1 + RS))

In [ ]:
bac['rsi'] = RSI(bac['Close'])

In [ ]:
# Calculate VWAP (Volume Weighted Average Price)
def VWAP(bac):
    q = bac['Volume']
    p = bac['Close']
    vwap = (p * q).cumsum() / q.cumsum()
    return vwap

In [ ]:
bac['vwap'] = VWAP(bac)


In [ ]:
# Drop rows with NaN values created by rolling calculations
bac.dropna(inplace=True)

# Add Regressors and Fit the Prophet Model

In [ ]:
# create the Prophet model
bac_model = Prophet()

In [ ]:
# Add additional indicators as regressors
bac_model.add_regressor('SMA_50')
bac_model.add_regressor('SMA_200')
bac_model.add_regressor('ma50')
bac_model.add_regressor('ma200')
bac_model.add_regressor('rsi')
bac_model.add_regressor('vwap')

In [ ]:
#column check
print(bac.columns)

In [ ]:
bac.reset_index(drop=True, inplace=True)

Fit the model

In [ ]:
# fit the model
bac_model.fit(bac[['ds', 'y', 'ma50', 'ma200', 'rsi', 'vwap', 'SMA_50', 'SMA_200']])

In [ ]:
# Create a future dataframe for predictions
bac_future = bac_model.make_future_dataframe(periods=365)

To make predictions, you need future values of the indicators
For this example, we assume that the future values are calculated similarly, and added to the future dataframe.
Here you should add actual future values for the indicators

In [ ]:
# Example (future values need to be calculated or assumed):
bac_future['ma50'] = bac['ma50'].iloc[-1]
bac_future['ma200'] = bac['ma200'].iloc[-1]
bac_future['rsi'] = bac['rsi'].iloc[-1]
bac_future['vwap'] = bac['vwap'].iloc[-1]
bac_future['SMA_50'] = bac['SMA_50'].iloc[-1]
bac_future['SMA_200'] = bac['SMA_200'].iloc[-1]

In [ ]:
# Predict future values
bac_forecast = bac_model.predict(bac_future)

In [ ]:
# Plot the forecast
bac_model.plot(bac_forecast)

Reflecting on the above plot, the [black] points are the actual adjusted closing price, 
the [thin blue] lines are the models past price predictions, 
the [dark blue] is the future forcast, as we move further out, our model loses confidence in its accuracy

create interactive plots

In [ ]:
#import library
import nbformat
print(nbformat.__version__)

In [ ]:
#chart an interactive plot
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(bac_model, bac_forecast)

Let's explore the chart above:
* Our horizontal (`x`) axis is how many days into the future we're predicting. That's our **horizon**
* Our vertical (`y`) axis is the **Mean Absolute Error** - the average error in our predictions in stock price.


# Model evaluation

Let's use the [Diagnostics](https://facebook.github.io/prophet/docs/diagnostics.html) library from `prophet` to validate our model using `cross_validation`.

In [ ]:
#import library
from prophet.diagnostics import cross_validation

Now **create a DataFrame that is the result of running `cross_validation` on our model, with a horizon of 180, 80 or 60 days! 

In [ ]:
bac_validation = cross_validation(bac_model, period='60 days', horizon='80 days')

In [ ]:
bac_validation

# Interpretation

In [ ]:
bac_model.plot(bac_forecast);

* We can see from the prediction `plot` that we have a point after which the model quickly starts to lose confidence.
* We can also see the same from the errors - as we try to predict further into the future, our accuracy goes down.


**Finally, let's visualize the errors** (differences) that between our model prediction and the seen reality. We will use the `mae` (Mean Absolute Error) as the metric. 

In [ ]:
from prophet.plot import plot_cross_validation_metric
bac_2 = plot_cross_validation_metric(bac_validation, metric='mae')

**Our best case interpretation:**

We can see some clear highs and lows in our errors, which is not good. The main reason for that is that our data is `monthly` but we are doing our cross-validation `by day`. 


The **volatility** (waviness) or errors are fairly stable with some certainty. 